In [4]:
# for iterating over pngs
import os
import subprocess

# for conversion
from typing import Tuple, List, Union
from skimage import io
#from scikit-image import io
import SimpleITK as sitk
import numpy as np
import tifffile

from skimage.color import rgb2gray
from skimage import data


def convert_2d_image_to_nifti(input_filename: str, output_filename_truncated: str, spacing=(999, 1, 1),
                              transform=None, is_seg: bool = False) -> None:
    """
    Reads an image (must be a format that it recognized by skimage.io.imread) and converts it into a series of niftis.
    The image can have an arbitrary number of input channels which will be exported separately (_0000.nii.gz,
    _0001.nii.gz, etc for images and only .nii.gz for seg).
    Spacing can be ignored most of the time.
    !!!2D images are often natural images which do not have a voxel spacing that could be used for resampling. These images
    must be resampled by you prior to converting them to nifti!!!

    Datasets converted with this utility can only be used with the 2d U-Net configuration of nnU-Net

    If Transform is not None it will be applied to the image after loading.

    Segmentations will be converted to np.uint32!

    :param is_seg:
    :param transform:
    :param input_filename:
    :param output_filename_truncated: do not use a file ending for this one! Example: output_name='./converted/image1'. This
    function will add the suffix (_0000) and file ending (.nii.gz) for you.
    :param spacing:
    :return:
    """
    img = io.imread(input_filename)

    if transform is not None:
        img = transform(img)

    if len(img.shape) == 2:  # 2d image with no color channels
        img = img[None, None]  # add dimensions
    else:
        assert len(img.shape) == 3, "image should be 3d with color channel last but has shape %s" % str(img.shape)
        # we assume that the color channel is the last dimension. Transpose it to be in first
        img = img.transpose((2, 0, 1))
        # add third dimension
        img = img[:, None]

    # image is now (c, x, x, z) where x=1 since it's 2d
   # if is_seg:
    #    assert img.shape[0] == 1, 'segmentations can only have one color channel, not sure what happened here'

    
    for j, i in enumerate(img[0:1]):

        if is_seg:
            i = i.astype(np.uint32)

        itk_img = sitk.GetImageFromArray(i)
        itk_img.SetSpacing(list(spacing)[::-1])
        if not is_seg:
            sitk.WriteImage(itk_img, output_filename_truncated + "_%04.0d.nii.gz" % j)
        else:
            sitk.WriteImage(itk_img, output_filename_truncated + ".nii.gz")

# Specify the folder containing PNG files
folder_path = '/Users/seena/Downloads/OGImages'

# List all files in the folder
png_files = [file for file in os.listdir(folder_path) if file.endswith('.png')]

# Iterate through each PNG file
for png_file in png_files:
    # Generate full path for the input and output filenames
    input_filepath = os.path.join(folder_path, png_file)
    output_filename = os.path.splitext(png_file)[0]  # Remove the .png extension
    output_filepath = os.path.join(folder_path, output_filename)

    # Run the specified command
    convert_2d_image_to_nifti(input_filepath, output_filepath)

    print(f"Conversion complete for {png_file}")


/Users/seena/miniforge3/envs/miniForge3Env/lib/python3.9/site-packages/imageio/plugins/pillow.py:297: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(


Conversion complete for 0400_ck023_OD_Scan_3.png.png
Conversion complete for 269_REPS030_OD_Scan_4.png.png
Conversion complete for 0377_ck021_OD_Scan_4.png.png
Conversion complete for 0109_N23_OD_Scan_0.png.png
Conversion complete for 0149_HVS016_OS_Scan_4.png.png
Conversion complete for 0192_N9_OS_Scan_5.png.png
Conversion complete for 0190_N9_OS_Scan_3.png.png
Conversion complete for 0256_ck009_OD_Scan_3.png.png
Conversion complete for 0141_HVS016_OD_Scan_2.png.png
Conversion complete for 0242_ck008_OD_Scan_1.png.png
Conversion complete for 0420_ck025_OD_Scan_5.png.png
Conversion complete for 0138_HVS015_OS_Scan_5.png.png
Conversion complete for 0630_ck049_OS_Scan_5.png.png
Conversion complete for 112_REPS014_OD_Scan_3.png.png
Conversion complete for 0062_N15_OD_Scan_1.png.png
Conversion complete for 0130_HVS015_OD_Scan_3.png.png
Conversion complete for 0041_HVS007_OD_Scan_4.png.png
Conversion complete for 0002_HVS001_OS_Scan_1.png.png
Conversion complete for 0071_HVS010_OD_Scan_4.pn

In [6]:
import shutil

def group_image_files(file_extension,folder_path):
    # Create subfolders for different file extensions
    image_folder_substring = file_extension + "_files"
    image_folder = os.path.join(folder_path, image_folder_substring)
    nifti_folder = os.path.join(folder_path, 'nifti_files')

    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(nifti_folder, exist_ok=True)

    # List all files in the folder
    all_files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

    # Group files based on file extension
    for file in all_files:
        file_path = os.path.join(folder_path, file)
        entire_file_extension = "." + file_extension
        if file.lower().endswith(entire_file_extension):
            # Move image files to the image_files subfolder
            shutil.move(file_path, os.path.join(image_folder, file))
            print(f"Moved {file} to image_folder subfolder")
        else:
            # Move nifti files to the nifti_files subfolder
            shutil.move(file_path, os.path.join(nifti_folder, file))
            print(f"Moved {file} to nifti_files subfolder")


folder_path = '/Users/seena/Downloads/OGImages'
group_image_files(folder_path)


Moved 97_REPS013_OD_Scan_0.png_0000.nii.gz to other_files subfolder
Moved 0400_ck023_OD_Scan_3.png.png to png_files subfolder
Moved 0099_N22_OD_Scan_2.png_0000.nii.gz to other_files subfolder
Moved 269_REPS030_OD_Scan_4.png.png to png_files subfolder
Moved 64_REPS007_OS_Scan_3.png_0000.nii.gz to other_files subfolder
Moved 0377_ck021_OD_Scan_4.png.png to png_files subfolder
Moved 0109_N23_OD_Scan_0.png.png to png_files subfolder
Moved 0149_HVS016_OS_Scan_4.png.png to png_files subfolder
Moved 0192_N9_OS_Scan_5.png.png to png_files subfolder
Moved 0190_N9_OS_Scan_3.png.png to png_files subfolder
Moved 0256_ck009_OD_Scan_3.png.png to png_files subfolder
Moved 0141_HVS016_OD_Scan_2.png.png to png_files subfolder
Moved 0179_HVS020_OD_Scan_4.png_0000.nii.gz to other_files subfolder
Moved 389_REPS043_OS_Scan_4.png_0000.nii.gz to other_files subfolder
Moved 0242_ck008_OD_Scan_1.png.png to png_files subfolder
Moved 0677_ck053_OS_Scan_4.png_0000.nii.gz to other_files subfolder
Moved 0420_ck025_